In [ ]:
from pytorch_forecasting.data.examples import generate_ar_data
import matplotlib.pyplot as plt
import pandas as pd
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
import pytorch_lightning as pl
from pytorch_forecasting import NegativeBinomialDistributionLoss, DeepAR
import torch
from pytorch_forecasting.data.encoders import TorchNormalizer


In [6]:
data = pd.read_csv('1_f_nbinom_train.csv')


,series,time_idx,value
0,0,0,0
1,0,1,0
2,0,2,0
3,0,3,12
4,0,4,0
...,...,...,...
5014,0,5014,0
5015,0,5015,12
5016,0,5016,0
5017,0,5017,0


In [9]:
data["date"] = pd.Timestamp("2021-08-24") + pd.to_timedelta(data.time_idx, "H")


data['_hour_of_day'] = str(data["date"].dt.hour)
data['_day_of_week'] = str(data["date"].dt.dayofweek)
data['_day_of_month'] = str(data["date"].dt.day)
data['_day_of_year'] = str(data["date"].dt.dayofyear)
data['_week_of_year'] = str(data["date"].dt.weekofyear)
data['_month_of_year'] = str(data["date"].dt.month)
data['_year'] = str(data["date"].dt.year)

data['value'] = data['value'].astype(float)
print(type(data['value'][0])) 
print(len(data.iloc[0:-620]))



<class 'numpy.float64'>
4399


C:\Users\josepeeterson.er\AppData\Local\Temp\ipykernel_22536\2062568536.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data['_week_of_year'] = str(data["date"].dt.weekofyear)


In [64]:
max_encoder_length = 60
max_prediction_length = 20
training_cutoff = data["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    data.iloc[0:-620],
    time_idx="time_idx",
    target="value",
    categorical_encoders={"series": NaNLabelEncoder(add_nan=True).fit(data.series), "_hour_of_day": NaNLabelEncoder(add_nan=True).fit(data._hour_of_day), \
       "_day_of_week": NaNLabelEncoder(add_nan=True).fit(data._day_of_week), "_day_of_month" : NaNLabelEncoder(add_nan=True).fit(data._day_of_month), "_day_of_year" : NaNLabelEncoder(add_nan=True).fit(data._day_of_year), \
        "_week_of_year": NaNLabelEncoder(add_nan=True).fit(data._week_of_year), "_year": NaNLabelEncoder(add_nan=True).fit(data._year)},
    group_ids=["series"],
    min_encoder_length=max_encoder_length,
    max_encoder_length=max_encoder_length,
    min_prediction_length=max_prediction_length,
    max_prediction_length=max_prediction_length,
    time_varying_unknown_reals=["value"],
    time_varying_known_categoricals=["_hour_of_day","_day_of_week","_day_of_month","_day_of_year","_week_of_year","_year" ],
    time_varying_known_reals=["time_idx"],
    add_relative_time_idx=False,
    randomize_length=None,
    scalers=[],
    target_normalizer=TorchNormalizer(method="identity",center=False,transformation=None )

)

In [5]:
validation = TimeSeriesDataSet.from_dataset(
    training,
    data.iloc[-620:-420],
    # predict=True,
    stop_randomization=True,
)       

In [6]:
batch_size = 64
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=8)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=8)


In [7]:
# save datasets
training.save("training.pkl")
validation.save("validation.pkl")

In [8]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=5, verbose=False, mode="min")
lr_logger = LearningRateMonitor()


In [9]:
trainer = pl.Trainer(
    max_epochs=1,
    gpus=0,
    gradient_clip_val=0.1,
    limit_train_batches=30,
    limit_val_batches=3,
    # fast_dev_run=True,
    # logger=logger,
    # profiler=True,
    callbacks=[lr_logger, early_stop_callback],
)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
deepar = DeepAR.from_dataset(
    training,
    learning_rate=0.1,
    hidden_size=32,
    dropout=0.1,
    loss=NegativeBinomialDistributionLoss(),
    log_interval=10,
    log_val_interval=3,
    # reduce_on_plateau_patience=3,
)
print(f"Number of parameters in network: {deepar.size()/1e3:.1f}k")


In [11]:
# #find optimal learning rate
# deepar.hparams.log_interval = -1
# deepar.hparams.log_val_interval = -1
# trainer.limit_train_batches = 1.0
# res = trainer.tuner.lr_find(
#     deepar, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader, min_lr=1e-5, max_lr=1e2
# )

In [12]:
torch.set_num_threads(10)
trainer.fit(
    deepar,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)


  | Name                   | Type                             | Params
----------------------------------------------------------------------------
0 | loss                   | NegativeBinomialDistributionLoss | 0     
1 | logging_metrics        | ModuleList                       | 0     
2 | embeddings             | MultiEmbedding                   | 12    
3 | rnn                    | LSTM                             | 13.8 K
4 | distribution_projector | Linear                           | 66    
----------------------------------------------------------------------------
13.9 K    Trainable params
0         Non-trainable params
13.9 K    Total params
0.056     Total estimated model params size (MB)


c:\Users\josepeeterson.er\Miniconda3\envs\pytorch\lib\site-packages\pytorch_lightning\trainer\data_loading.py:428: UserWarning: The number of training samples (30) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0: 100%|██████████| 32/32 [01:36<00:00,  3.03s/it, loss=1.92, v_num=17, train_loss_step=1.550, val_loss=1.590, train_loss_epoch=2.190]


In [18]:
# calcualte mean absolute error on validation set
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])

print(actuals.shape)

predictions = deepar.predict(data=val_dataloader,mode='prediction',return_index=True,num_workers=8,show_progress_bar=True)



print(f"Mean absolute error of model: {(actuals - torch.tensor(predictions)).abs().mean()}")


torch.Size([121, 20])


Predict: 100%|██████████| 2/2 [00:24<00:00, 12.02s/ batches]


KeyError: 0

In [23]:
print(actuals.shape)
print(len(predictions[0][0]))



torch.Size([121, 20])
20
